In [ ]:
!pip install transformers
!pip install seqeval
!pip install sentencepiece
import sentencepiece as spm
from seqeval.metrics import precision_score as seq_precision, recall_score as seq_recall, f1_score as seq_f1
from transformers import AutoTokenizer, XLMRobertaModel, XLMRobertaForMaskedLM
import json
import logging
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import os
import torch
import numpy as np
import torch.nn as nn 
from torch.nn import functional as F
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
from easydict import EasyDict
import gc
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.optim import Adam
import pickle
logger = logging.getLogger(__name__)

     |████████████████████████████████| 1.5MB 17.3MB/s 
     |████████████████████████████████| 890kB 48.0MB/s 
     |████████████████████████████████| 2.9MB 52.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f44bb1430968e40358accc779fc96b07d2a9da0626685e427023eb67b46aa2a8
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=1409e13957dfea64146fa4a9e587ecaacd70b65f978027769f8dba6f37dda610
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval
     |████████████████████████████████| 1.2MB 15.1MB/s 


# Utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DataLoader for Roberta model

In [ ]:

def init_logger():
    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    
logger = logging.getLogger(__name__)



class InputExample(object):
    r"""
    One single example
    Args:
    example_id: str,  unique id for this example
    raw_text:list of words,  raw text contain misspelling words
    onehot_label:np array,  onehot array indicate position of the wrong words
    text_label:list of words,  ground true text label of raw_text
    """
    def __init__(self,example_id,  raw_text, onehot_labels= None, text_label= None):
        self.example_id = example_id
        self.raw_text = raw_text
        self.onehot_labels = onehot_labels
        self.text_label = text_label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

class InputFeatures(object):
    r"""
    Features of one single example
    Args:
    input_ids: ids of input raw text, after roberta tokenizer
    attention_mask: attention mask of input
    onehot_label: onehot array indicate position of wrong value

    """
    def __init__(self,input_ids, attention_mask, onehot_labels, output_ids ):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.onehot_labels = onehot_labels
        self.output_ids = output_ids
        
    def __repr__(self):
        return str(self.to_json_string())
    
    def to_dict(self):
        """Serializes this instanc eto a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output
    
    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent = 2, sort_keys = True) + '\n'

class JointProcessor(object):
    
    def __init__(self, args, ):
        self.args = args
        self.raw_text_file = 'raw_text.txt'
        self.onehot_labels_file = 'onehot_label.txt'
        self.text_label_file = 'text_label.txt'

    @classmethod
    # read the whole file into a list, each elements is a sentence
    def _read_file(cls, input_file, ):
        with open(input_file, 'r', encoding = 'utf-8') as f:
            lines = []
            for line in f.readlines():
                lines.append(line.rstrip())

            return lines

    def _create_examples(self, texts, onehot_labels, text_labels, set_type):
        """create examples for training and dev set"""

        examples = []
        for i , (text, onehot, text_label) in enumerate(zip(texts, onehot_labels, text_labels)):
            example_id = "%s-%s"%(set_type, i)
            # 1. input_raw_text
            words = text.split()
            # 2. onehot_label
            onehot = np.array([int(x) for x in onehot.split()], dtype = int)
            # 3. text label
            text_label = text_label.split()
            #assert len(words) == len(onehot) == len(text_label)
            examples.append(InputExample(example_id, words, onehot, text_label))
        return examples

    def get_examples(self,mode):
        """
        Args:
        mode: train, dev, test
        """
        data_path = os.path.join(self.args.data_dir,  )
        logger.info('Looking at {}'.format(data_path))
        return self._create_examples(texts = self._read_file(os.path.join(data_path, self.raw_text_file)),
                                    onehot_labels = self._read_file(os.path.join(data_path, self.onehot_labels_file)),
                                    text_labels = self._read_file(os.path.join(data_path, self.text_label_file)), set_type = mode)
        
        

def convert_examples_to_features(examples, max_seq_len, tokenizer , pad_token_label_id= 0):

    ignore_token = ' ?? '
    pad_token_id = 0
    features = []
    for ex_index, example in tqdm(enumerate(examples)):
        if ex_index % 10000 == 0:
            logger.info('Writing example %d of %d'%(ex_index + 1, len(examples)))
        # tokenize word by word
        raw_word_tokens = []
        onehot_labels = []
        text_label_tokens = []

        for raw_word, onehot, label_word in zip(example.raw_text, example.onehot_labels, example.text_label):

            raw_word_token = tokenizer.encode(raw_word, out_type = int)

            text_label_token = tokenizer.encode(label_word, out_type = int)
            # all the subtoken of word will have the same onehot label as word
            

            if len(raw_word_token) > len(text_label_token):
                text_label_token += [pad_token_id]*(len(raw_word_token) - len(text_label_token))

            if len(raw_word_token) < len(text_label_token):
                raw_word_token += [pad_token_id]*(len(text_label_token) - len(raw_word_token))

            onehot_labels.extend([int(onehot)]*len(raw_word_token))     


            raw_word_tokens.extend(raw_word_token)

            text_label_tokens.extend(text_label_token)
            

        assert len(onehot_labels) == len(raw_word_tokens) == len(text_label_tokens), 'word tokens len does not match one hot labels'


        input_ids = raw_word_tokens[:max_seq_len]
        onehot_labels = onehot_labels[:max_seq_len]
        output_ids = text_label_tokens[:max_seq_len]

        padding_len = (max_seq_len - len(input_ids))
        input_ids = input_ids + [pad_token_id]*padding_len
        onehot_labels = onehot_labels + [pad_token_label_id]*padding_len
        output_ids = output_ids + [pad_token_id]*padding_len
 

        assert len(input_ids) == max_seq_len, "input_ids Error with input length {} vs {}".format(len(input_ids), max_seq_len)
        assert len(onehot_labels) == max_seq_len, "onehot_labels Error with input length {} vs {}".format(len(onehot_labels), max_seq_len)
        


        if ex_index < 5:
            logger.info('Example %s'%example.example_id)
            logger.info('Input ids %s'%' '.join([str(x) for x in input_ids]))
            logger.info('One hot labels %s'%' '.join([str(x) for x in onehot_labels]))
            logger.info('Output ids %s'%' '.join([str(x) for x in output_ids]))


        features.append(InputFeatures(input_ids = input_ids,attention_mask = None, onehot_labels = onehot_labels, output_ids = output_ids ))


    return features

def load_and_cache_examples(args,tokenizer,  mode):
    processor = JointProcessor(args)
    # Loooking for cached file
    cached_features_file = os.path.join(args.data_dir,
                                        'cached_%s_%s_%s'%(mode, str(args.max_seq_len), 'hard_masked_data') )

    if os.path.exists(cached_features_file):
        logger.info('Loading cached features file from %s'%cached_features_file)
        features = torch.load(cached_features_file)

    else:
        # load raw data to InputFeatures
        logger.info('Loading data from %s'%args.data_dir)
        if mode == 'train':
            examples= processor.get_examples(mode = 'train')
        elif mode == 'dev':
            examples = processor.get_examples(mode = 'dev')
        elif mode == 'test':
            examples = processor.get_examples(mode = 'test')
        else:
            raise Exception('Only train, dev, test are accepted')

        pad_token_label_id = args.ignore_index
        features = convert_examples_to_features(examples, args.max_seq_len,tokenizer, pad_token_label_id)
        logger.info('Save features file to %s'%cached_features_file)
        torch.save(features, cached_features_file)
    
    #Convert features to tensordataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype = torch.long)
    all_onehot_labels = torch.tensor([f.onehot_labels for f in features], dtype = torch.long)
    all_output_ids = torch.tensor([f.output_ids for f in features], dtype= torch.long)

    dataset = torch.utils.data.TensorDataset(all_input_ids, all_onehot_labels, all_output_ids, )
    return dataset




# Model

In [ ]:

def num_parameters(parameters):
    num = 0
    for i in parameters:
        num += len(i)
    return num
class Detector(nn.Module):
    def __init__(self, input_dim,output_dim,  embedding_dim, num_layers, hidden_size):

        super(Detector, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.embedding_dim  = embedding_dim
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(num_embeddings = self.input_dim, embedding_dim = self.embedding_dim, )
        self.LSTM = nn.LSTM(input_size = self.embedding_dim, hidden_size= self.hidden_size, num_layers = self.num_layers, 
                            batch_first = True, dropout = 0.1, bidirectional = True)
        self.linear = nn.Linear(self.hidden_size*2, self.output_dim)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        emb = self.embedding(x)
        outputs, (h_n, h_c) = self.LSTM(emb)
        logits = self.linear(outputs)
        p = self.sigmoid(logits)
        return p

# class HardMasked(nn.Module):
#     def __init__(self, detector, MaskedLM, tokenzier):
#         super(HardMasked, self).__init__()
#         self.detector = detector
#         self.MaskedLM = MaskedLM
#         self.tokenizer = tokenizer

#     def forward(self, s):
#         input_ids = self.tokenizer.encode(s, out_type = int)
#         p = self.detector(torch.tensor([input_ids], dtype = torch.long))
#         preds = (p > 0.5).float()
#     def prepare_input(self, input_ids, preds, s):
#         s = s.plits()
#         tokens = self.tokenizer.id_to_piece(input_ids)
#         onehot_labels = []
#         for word in s:
#             t = ''
#             for token in in





# Trainer
1. Debug model

In [ ]:
class Trainer(object):
    def __init__(self,args, model,train_dataloader, val_dataloader = None, ):
        """
        Training class
        Args:
            model: SoftMasked model
            learning_rate: learning rate for training
            epochs: number of training epochs
            train_dataloader: Pytorch DataLoader for training
            num_step: max gradient update steps
            num_logging_steps: logging each time number of gradient update % this number == 0
            val_dataloader: Pytorch DataLoader for validation


        """
        self.args = args 
        self.model = model.to(self.args.device)
        self.train_dataloader = train_dataloader


        
        self.epochs = args.epochs
        self.num_steps = args.num_steps
        self.val_dataloader = val_dataloader
        self.warmup_steps = args.warmup_steps
        self.model_dir = args.model_dir
        self.num_batch = len(train_dataloader)
        self.save_steps = args.save_steps
        # Prepare optimizer and schedule (linear warmup and decay)
        #we just do not use weight decay to bias and layerNorm weights, for bias, in the formular of L2 regularization do not has bias term 

        self.detector_loss = nn.BCELoss()

        # Only train the detector

        print('Optimize ', self.num_parameters(self.model.parameters()))
        #self.optimizer = AdamW(optimized_parameters, lr=self.learning_rate,)
        self.optimizer = Adam(self.model.parameters(), lr = self.args.learning_rate)
        #self.scheduler = get_linear_schedule_with_warmup(self.optimizer, num_warmup_steps=self.warmup_steps, num_training_steps=self.num_steps)


    def train(self):
        
        self.optimizer.zero_grad()

        global_steps = 0
        avg_loss = []
        eval_f1 = []
        for epoch in range(self.epochs):
            logger.info('Epoch : %d'%epoch, )
            
            for i, batch in tqdm(enumerate(self.train_dataloader), desc = 'Training'):
                self.model.train()
                batch = tuple(t.to(self.args.device) for t in batch)
                input_ids = batch[0]
                onehot_labels = batch[1]
                output_ids = batch[2]

                detector_prob = self.model(input_ids)

                loss = self.detector_loss(detector_prob.squeeze(dim = -1), onehot_labels.float())

                if self.args.gradient_accumulation_steps > 1:
                    loss = loss/self.args.gradient_accumulation_steps               
                
                loss.backward(retain_graph=True)
                if (i + 1)%self.args.gradient_accumulation_steps == 0:
                    #torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)
                    self.optimizer.step()
                    self.optimizer.zero_grad()
                    global_steps += 1
                    avg_loss.append(float(loss)) 


                    if self.args.logging_steps > 0 and global_steps % self.args.logging_steps == 0:
                        if self.val_dataloader is not None:
                            eval_f1.append(self.eval())
                        logger.info('Training avg loss %f'%(np.mean(avg_loss)))
                        avg_loss = []

                    if self.args.save_steps > 0 and global_steps % self.args.save_steps == 0:
                        self.save_model()
                if 0<self.args.num_steps < global_steps:
                    logger.info('Training reach %i steps and is stopped'%global_steps)
                    return eval_f1
                
            if 0<self.args.num_steps < global_steps:
                logger.info('Training reach %i steps and is stopped'%global_steps)
                return eval_f1
                


    def eval(self,):
        logger.info('Running evaluation on dev set')

        eval_loss = []
        
        detector_preds = []

        detector_labels = []

        self.model.eval()
        for batch in tqdm(self.val_dataloader, 'Evaluating'):
            batch = tuple(t.to(self.args.device) for t in batch)
            with torch.no_grad():
                input_ids = batch[0]
                onehot_labels = batch[1]
                output_ids = batch[2]

                detector_probs = self.model(input_ids)


                loss = self.detector_loss(detector_probs.squeeze(dim = -1), onehot_labels.float())

           
                eval_loss.append(float(loss))

                detector_pred = (detector_probs.detach().cpu().numpy()  > 0.5 ).astype(int).squeeze(-1)        
                detector_preds.extend(detector_pred)


                detector_labels.extend( onehot_labels.detach().cpu().numpy()) 


        detector_labels = np.array(detector_labels)
        detector_preds = np.array(detector_preds)
        
        precision = precision_score(detector_labels,detector_preds , average = 'micro')
        recall = recall_score(detector_labels,detector_preds , average = 'micro' )
        f1 = f1_score(detector_labels,detector_preds, average = 'micro' )

        logger.info('Total loss mean: %f'%(np.mean(eval_loss)))

        logger.info('Detector Precision, Recall, F1:  %f, %f, %f'%(precision, recall, f1))
        return f1

    def save_model(self):
        # Save model checkpoint (Overwrite)
        if not os.path.exists(self.args.model_dir):
            os.makedirs(self.args.model_dir)

        torch.save(self.model, os.path.join(self.args.model_dir, 'Detector.pkl'))

        logger.info("Saving model checkpoint to %s", self.model_dir)

    def num_parameters(self,parameters):
        return  sum(p.numel() for p in parameters)


# Main

In [ ]:
init_logger()
tokenizer_path = '/content/drive/MyDrive/nlp projects/Text_correction/spm_tokenizer.model'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = spm.SentencePieceProcessor(tokenizer_path)


data_dir = '/content/drive/MyDrive/nlp projects/Text_correction/all_data/train_data/train1'
args = EasyDict({'data_dir': data_dir, 'ignore_index': 0, 'max_seq_len': 100, })

train1 = load_and_cache_examples(args = args, tokenizer = tokenizer, mode = 'train' )

data_dir = '/content/drive/MyDrive/nlp projects/Text_correction/all_data/train_data/train2'
args = EasyDict({'data_dir': data_dir, 'ignore_index': 0, 'max_seq_len': 100, })

train2 = load_and_cache_examples(args = args, tokenizer = tokenizer, mode = 'train' )


train_data = torch.utils.data.ConcatDataset([train1, train2])

In [ ]:
data_dir = '/content/drive/MyDrive/nlp projects/Text_correction/all_data/dev_data'
args = EasyDict({'data_dir': data_dir, 'ignore_index': 0, 'max_seq_len': 100, })

dev_data = load_and_cache_examples(args = args, tokenizer= tokenizer, mode = 'dev' )



01/13/2021 04:20:37 - INFO - __main__ -   Loading cached features file from /content/drive/MyDrive/nlp projects/Text_correction/all_data/dev_data/cached_dev_100_hard_masked_data


In [ ]:
train_sampler = RandomSampler(train_data)
dev_sampler = SequentialSampler(dev_data)
# test_sampler = SequentialSampler(test_dataset)


train_dataloader = DataLoader(dataset = train_data, batch_size = 128, sampler = train_sampler,)
dev_dataloader = DataLoader(dataset = dev_data, batch_size = 128, sampler = dev_sampler)
# test_dataloader = DataLoader(dataset = test_dataset, batch_size = 32, sampler = test_sampler )

In [ ]:
detector = Detector(input_dim = 10000,output_dim = 1,  embedding_dim = 512, num_layers = 2, hidden_size = 768)

In [ ]:
model = torch.load('/content/drive/MyDrive/nlp projects/Text_correction/all_data/train_data/Detector.pkl')

In [ ]:
model_dir = '/content/drive/MyDrive/nlp projects/Text_correction/all_data'
training_args = EasyDict({'model_dir': model_dir, 
                          'learning_rate': 0.0005, 
                          'epochs': 2,          # loop over this number of epochs until reach num_steps 
                          'num_steps':200000, # total number of training steps
                          'logging_steps': 2000, # do eval each time reach this number of steps
                          'save_steps': 2000,  #save model each time reach this training steps
                          'device': device,
                
                            'warmup_steps': 0,
                          'gradient_accumulation_steps': 1, # update parameters after this number of gradient accumulation steps
                          'max_seq_len': 100, 
                          'vocab_size': 10000
                          })
trainer = Trainer(args = training_args, model = model, 
                  train_dataloader = train_dataloader,val_dataloader = dev_dataloader )

Optimize  15623169


In [ ]:

f1_eval=trainer.train()



01/13/2021 09:40:26 - INFO - __main__ -   Epoch : 0


01/13/2021 09:49:12 - INFO - __main__ -   Running evaluation on dev set


01/13/2021 09:49:28 - INFO - __main__ -   Total loss mean: 0.015273
01/13/2021 09:49:28 - INFO - __main__ -   Detector Precision, Recall, F1:  0.969618, 0.957650, 0.963597
01/13/2021 09:49:28 - INFO - __main__ -   Training avg loss 0.012892
01/13/2021 09:49:29 - INFO - __main__ -   Saving model checkpoint to /content/drive/MyDrive/nlp projects/Text_correction/all_data
01/13/2021 09:58:14 - INFO - __main__ -   Running evaluation on dev set


01/13/2021 09:58:30 - INFO - __main__ -   Total loss mean: 0.014786
01/13/2021 09:58:30 - INFO - __main__ -   Detector Precision, Recall, F1:  0.971362, 0.958383, 0.964829
01/13/2021 09:58:30 - INFO - __main__ -   Training avg loss 0.012419
01/13/2021 09:58:30 - INFO - __main__ -   Saving model checkpoint to /content/drive/MyDrive/nlp projects/Text_correction/all_data
01/13/2021 10:07:17 - INFO - __main__ -   Running evaluation on dev set


01/13/2021 10:07:33 - INFO - __main__ -   Total loss mean: 0.014415
01/13/2021 10:07:33 - INFO - __main__ -   Detector Precision, Recall, F1:  0.969004, 0.962377, 0.965679
01/13/2021 10:07:33 - INFO - __main__ -   Training avg loss 0.012381
01/13/2021 10:07:33 - INFO - __main__ -   Saving model checkpoint to /content/drive/MyDrive/nlp projects/Text_correction/all_data
01/13/2021 10:16:19 - INFO - __main__ -   Running evaluation on dev set


...

01/13/2021 10:52:27 - INFO - __main__ -   Running evaluation on dev set


01/13/2021 10:52:43 - INFO - __main__ -   Total loss mean: 0.014110
01/13/2021 10:52:43 - INFO - __main__ -   Detector Precision, Recall, F1:  0.973439, 0.961681, 0.967525
01/13/2021 10:52:43 - INFO - __main__ -   Training avg loss 0.011166
01/13/2021 10:52:43 - INFO - __main__ -   Saving model checkpoint to /content/drive/MyDrive/nlp projects/Text_correction/all_data
01/13/2021 11:01:29 - INFO - __main__ -   Running evaluation on dev set


01/13/2021 11:01:45 - INFO - __main__ -   Total loss mean: 0.014153
01/13/2021 11:01:45 - INFO - __main__ -   Detector Precision, Recall, F1:  0.970817, 0.964085, 0.967439
01/13/2021 11:01:45 - INFO - __main__ -   Training avg loss 0.009000
01/13/2021 11:01:45 - INFO - __main__ -   Saving model checkpoint to /content/drive/MyDrive/nlp projects/Text_correction/all_data
01/13/2021 11:10:32 - INFO - __main__ -   Running evaluation on dev set


01/13/2021 11:10:47 - INFO - __main__ -   Total loss mean: 0.013919
01/13/2021 11:10:47 - INFO - __main__ -   Detector Precision, Recall, F1:  0.972914, 0.962788, 0.967824
01/13/2021 11:10:47 - INFO - __main__ -   Training avg loss 0.009278
01/13/2021 11:10:48 - INFO - __main__ -   Saving model checkpoint to /content/drive/MyDrive/nlp projects/Text_correction/all_data


KeyboardInterrupt: ignored